# Machine Learning Foundation

## Course 5, Part g: RNN DEMO

## Using RNNs to classify sentiment on IMDB data
- **For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.**
- **Our data consists of 25000 training sequences and 25000 test sequences.**
- **The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.**

- Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).
- This will save us a lot of the drudgery that is usually involved when working with raw text.

- We will walk through the preparation of the data and the building of an RNN model.
- Then it will be your turn to build your own models (and prepare the data how you see fit).

In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
max_features = 20000 # This is used in loading the data, picks the most common (max_features) words
maxlen = 30 # Maximum length of a sequence - truncate after this
batch_size = 32

In [3]:
# Load in the data.
# The function automatically tokenizes the text into distinct integers.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=max_features)
print(f"{len(x_train)} train sequences")
print(f"{len(x_test)} test sequences")

17465344/17464789 [==============================] - 11s 1us/step


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
C:\Users\batuh\anaconda3\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])


25000 train sequences
25000 test sequences


C:\Users\batuh\anaconda3\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [8]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)
print(f"x_train shape: {x_train.shape}")
print(f"x_test shape: {x_test.shape}")

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [9]:
x_train[123, :]

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99])

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`tf.keras.layers.Embedding(
    input_dim,
    output_dim,
    embeddings_initializer="uniform",
    embeddings_regularizer=None,
    activity_regularizer=None,
    embeddings_constraint=None,
    mask_zero=False,
    input_length=None,
    **kwargs
)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`tf.keras.layers.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  
    - Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.
    - So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.

In [11]:
# Let's build a RNN
rnn_hidden_dim = 5
word_embedding_dim = 50

model_rnn = tf.keras.models.Sequential()
model_rnn.add(tf.keras.layers.Embedding(max_features, word_embedding_dim)) #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(tf.keras.layers.SimpleRNN(rnn_hidden_dim,
                                       kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
                                       recurrent_initializer=tf.keras.initializers.Identity(gain=1.0),
                                       activation="relu",
                                       input_shape=x_train.shape[1:]))
model_rnn.add(tf.keras.layers.Dense(1, activation="sigmoid"))

In [12]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 50)          1000000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 5)                 280       
_________________________________________________________________
dense (Dense)                (None, 1)                 6         
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [13]:
rmsprop = tf.keras.optimizers.RMSprop(learning_rate=0.0001)

model_rnn.compile(loss="binary_crossentropy",
                  optimizer=rmsprop,
                  metrics=["accuracy"])

In [14]:
model_rnn.fit(x_train, y_train, batch_size=batch_size,
              epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 7s 8ms/step - loss: 0.6537 - accuracy: 0.6278 - val_loss: 0.5925 - val_accuracy: 0.6900
Epoch 2/10
782/782 [==============================] - 6s 7ms/step - loss: 0.5457 - accuracy: 0.7224 - val_loss: 0.5405 - val_accuracy: 0.7215
Epoch 3/10
782/782 [==============================] - 6s 7ms/step - loss: 0.4866 - accuracy: 0.7666 - val_loss: 0.5014 - val_accuracy: 0.7491
Epoch 4/10
782/782 [==============================] - 6s 7ms/step - loss: 0.4473 - accuracy: 0.7918 - val_loss: 0.4800 - val_accuracy: 0.7643
Epoch 5/10
782/782 [==============================] - 6s 7ms/step - loss: 0.4200 - accuracy: 0.8064 - val_loss: 0.4666 - val_accuracy: 0.7725
Epoch 6/10
782/782 [==============================] - 6s 7ms/step - loss: 0.4003 - accuracy: 0.8190 - val_loss: 0.4580 - val_accuracy: 0.7794
Epoch 7/10
782/782 [==============================] - 6s 8ms/step - loss: 0.3853 - accuracy: 0.8264 - val_loss: 0.4509 - val_accuracy: 0.7853
Epoch 

In [15]:
score, acc = model_rnn.evaluate(x_test, y_test,
                                batch_size=batch_size)
print(f"Test score: {score}")
print(f"Test accuracy: {acc}")

782/782 [==============================] - 1s 1ms/step - loss: 0.4496 - accuracy: 0.7893
Test score: 0.44961902499198914
Test accuracy: 0.7893199920654297


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?

In [16]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=max_features)

x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
C:\Users\batuh\anaconda3\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\batuh\anaconda3\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant

In [18]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = tf.keras.models.Sequential()
model_rnn.add(tf.keras.layers.Embedding(max_features, word_embedding_dim)) #This layer takes each integer in the sequence
model_rnn.add(tf.keras.layers.SimpleRNN(rnn_hidden_dim,
                                    kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
                                    recurrent_initializer=tf.keras.initializers.Identity(gain=1.0),
                                    activation='relu',
                                    input_shape=x_train.shape[1:]))
model_rnn.add(tf.keras.layers.Dense(1, activation="sigmoid"))

In [20]:
rmsprop = tf.keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss="binary_crossentropy",
                  optimizer=rmsprop,
                  metrics=["accuracy"])

In [21]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 13s 16ms/step - loss: 0.6400 - accuracy: 0.6364 - val_loss: 0.5515 - val_accuracy: 0.7299
Epoch 2/10
782/782 [==============================] - 12s 15ms/step - loss: 0.4941 - accuracy: 0.7615 - val_loss: 0.5046 - val_accuracy: 0.7542
Epoch 3/10
782/782 [==============================] - 12s 15ms/step - loss: 0.4349 - accuracy: 0.8002 - val_loss: 0.4671 - val_accuracy: 0.7813
Epoch 4/10
782/782 [==============================] - 12s 15ms/step - loss: 0.3942 - accuracy: 0.8232 - val_loss: 0.4421 - val_accuracy: 0.7922
Epoch 5/10
782/782 [==============================] - 12s 15ms/step - loss: 0.3629 - accuracy: 0.8415 - val_loss: 0.4266 - val_accuracy: 0.8014
Epoch 6/10
782/782 [==============================] - 12s 15ms/step - loss: 0.3383 - accuracy: 0.8552 - val_loss: 0.4069 - val_accuracy: 0.8140
Epoch 7/10
782/782 [==============================] - 12s 15ms/step - loss: 0.3176 - accuracy: 0.8654 - val_loss: 0.4001 - val_accuracy:

***

In [22]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = tf.keras.models.Sequential()
model_rnn.add(tf.keras.layers.Embedding(max_features, word_embedding_dim)) #This layer takes each integer in the sequence
model_rnn.add(tf.keras.layers.SimpleRNN(rnn_hidden_dim,
                                    kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.001),
                                    recurrent_initializer=tf.keras.initializers.Identity(gain=1.0),
                                    activation='relu',
                                    input_shape=x_train.shape[1:]))
model_rnn.add(tf.keras.layers.Dense(1, activation="sigmoid"))

In [23]:
rmsprop = tf.keras.optimizers.RMSprop(learning_rate = .0001)

model_rnn.compile(loss="binary_crossentropy",
                  optimizer=rmsprop,
                  metrics=["accuracy"])

In [24]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 12s 14ms/step - loss: 0.6458 - accuracy: 0.6299 - val_loss: 0.5710 - val_accuracy: 0.7117
Epoch 2/10
782/782 [==============================] - 11s 14ms/step - loss: 0.5139 - accuracy: 0.7497 - val_loss: 0.5011 - val_accuracy: 0.7589
Epoch 3/10
782/782 [==============================] - 11s 14ms/step - loss: 0.4495 - accuracy: 0.7909 - val_loss: 0.4698 - val_accuracy: 0.7770
Epoch 4/10
782/782 [==============================] - 11s 14ms/step - loss: 0.4065 - accuracy: 0.8181 - val_loss: 0.4427 - val_accuracy: 0.7923
Epoch 5/10
782/782 [==============================] - 11s 14ms/step - loss: 0.3720 - accuracy: 0.8392 - val_loss: 0.4220 - val_accuracy: 0.8021
Epoch 6/10
782/782 [==============================] - 11s 14ms/step - loss: 0.3453 - accuracy: 0.8537 - val_loss: 0.4040 - val_accuracy: 0.8151
Epoch 7/10
782/782 [==============================] - 11s 14ms/step - loss: 0.3257 - accuracy: 0.8622 - val_loss: 0.4089 - val_accuracy: